<a href="https://colab.research.google.com/github/joomj2000/DeepLearning/blob/main/fashionCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install matplotlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader

In [3]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
train_dataset=torchvision.datasets.FashionMNIST("/data", download=True,transform=transforms.Compose([transforms.ToTensor()]))
test_dataset=torchvision.datasets.FashionMNIST("/data", download=True,transform=transforms.Compose([transforms.ToTensor()]))

In [20]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=100)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=100)

In [21]:
labels_map={0:'T-Shirt',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',5:'Sandal',6:'Shirt',7:'Sneaker',8:'Bag',9:'Ankle Boot'}

In [43]:
import torch
import torch.nn as nn

class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 14x14
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  #6x6
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  #2x2
        )

        self.fc1 = nn.Linear(in_features=32*2*2, out_features=600)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)

    def forward(self, input_data):
        out = self.layer1(input_data)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [44]:
learning_rate=0.001;
model=FashionCNN();
model.to(device)
criterion=nn.CrossEntropyLoss();
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate);
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=128, out_features=600, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_fe

In [45]:
num_epochs=5
count=0
loss_list=[]
iteration_list=[]
accuracy_list=[]

predictions_list=[]
labels_list=[]

for epoch in range(num_epochs):
    for images,labels in train_loader:
        images,labels =images.to(device), labels.to(device)
        train=images.view(100,1,28,28)
        outputs=model(train)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1

        if not (count%50):
            total=0
            correct=0
            for images,labels in test_loader:
                images,labels =images.to(device), labels.to(device)
                test=images.view(100,1,28,28)
                outputs=model(test)
                predictions=torch.max(outputs,1)[1].to(device)
                predictions_list.append(predictions)
                correct+=(predictions==labels).sum()
                total+=len(labels)
            accuracy=correct*100/total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        if not (count%500):
            print("iteration: {}, Loss :{}, Accracy:{}%".format(count,loss.data,accuracy))

iteration: 500, Loss :0.4060674011707306, Accracy:88.99666595458984%
iteration: 1000, Loss :0.3179017901420593, Accracy:90.75666809082031%
iteration: 1500, Loss :0.20137684047222137, Accracy:90.3550033569336%
iteration: 2000, Loss :0.19184261560440063, Accracy:92.33333587646484%
iteration: 2500, Loss :0.1484459787607193, Accracy:92.63666534423828%
iteration: 3000, Loss :0.1722625344991684, Accracy:92.57500457763672%
